### Importar librerías necesarias

In [1]:
import itertools
import numpy as np
import pandas as pd
import regex as re
import selenium.webdriver as webdriver
import time

from geopy.geocoders import Nominatim
from geopy import distance
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

### Generar todas las combinaciones de direcciones (no correr)

In [17]:
# direcciones = [
#     "Brito del pino 1491 Montevideo Uruguay",
#     "Eulogio Caballero, Montevideo Uruguay",
#     "19 de abril 1213, Montevideo Uruguay",
#     "Copacabana 6808, Montevideo Uruguay",
#     "Galicia 1224, Montevideo Uruguay",
#     "Doctor Prudencio de Pena, Montevideo Uruguay",
#     "Rambla República de México, Montevideo Uruguay",
#     "Ricaldoni, Montevideo Uruguay",
#     "Doctor Alberto Zubiría, Montevideo Uruguay",
#     "Adolfo berro, Montevideo Uruguay",
#     "Los Nogales, Montevideo Uruguay",
#     "Paysandú, Montevideo, Uruguay",
#     "Himalaya, Montevideo, Uruguay",
#     "Carlos Tellier, Montevideo, Uruguay",
#     "Cooper, Montevideo, Uruguay",
#     "Oyarvide, Montevideo, Uruguay",
#     "Avenida Sarmiento, Montevideo, Uruguay",
#     "Líber Arce, Montevideo, Uruguay",
#     "Hocquart, Montevideo, Uruguay",
#     "Ignacio Barrios, Montevideo, Uruguay",
#     "Escalada, Montevideo, Uruguay",
#     "Fernando Torres, Montevideo, Uruguay",
#     "Carlos Reyles, Montevideo, Uruguay",
#     "Rizal, Montevideo, Uruguay",
#     "Chamizo, Montevideo, Uruguay"
# ]
# ### Chequear que las localizaciones puedan ser leídas por el Geolocalizador:
# geolocator = Nominatim(user_agent="sanico")
# for i in range(0,len(direcciones)):
#     print(geolocator.geocode(direcciones[i]))
    
# ### Generar combinaciones
# combinations = [list(x) for x in itertools.combinations(direcciones, 2)]

# ### Guardar en pandas dataframe y convertir a .csv
# combinations = pd.DataFrame(combinations, columns = ["retiro", "entrega"])
# print(combinations.shape)
# display(combinations.head())
# combinations.to_csv("combinations_address.csv", index=False)

(300, 2)


,retiro,entrega
0,Brito del pino 1491 Montevideo Uruguay,"Eulogio Caballero, Montevideo Uruguay"
1,Brito del pino 1491 Montevideo Uruguay,"19 de abril 1213, Montevideo Uruguay"
2,Brito del pino 1491 Montevideo Uruguay,"Copacabana 6808, Montevideo Uruguay"
3,Brito del pino 1491 Montevideo Uruguay,"Galicia 1224, Montevideo Uruguay"
4,Brito del pino 1491 Montevideo Uruguay,"Doctor Prudencio de Pena, Montevideo Uruguay"


### Extracción de datos: Función de web scrapping 

In [18]:
def get_results(search):

    url = "https://www.envigo.uy"
    browser = webdriver.Chrome("./chromedriver")
    browser.get(url)
    timeout = 15

    ### Clickear en cruz para quitar pop-up inicial:
    time.sleep(0.5)
    browser.find_element_by_xpath("//div[@id='modal-content']/div/button").click()

    ### Ingresar dirección de retiro:
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.ID, 'estimate_pickup_address'))
    )
    retiro = browser.find_element_by_id("estimate_pickup_address")
    retiro.send_keys(search['retiro'])
    
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    browser.find_element_by_xpath("//div[@class='pac-container pac-logo hdpi']/div[1]").click()

    ### Ingresar dirección de entrega:
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.ID, 'estimate_delivery_address'))
    )
    entrega = browser.find_element_by_id("estimate_delivery_address")
    entrega.send_keys(search['entrega'])
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    browser.find_element_by_xpath("//div[@class='pac-container pac-logo hdpi']/div[1]").click()
    time.sleep(1)
    ### Latitudes y longitudes

    # lat_retiro = browser.find_element_by_id("estimate_pickup_lat").get_attribute('value')
    # lng_retiro = browser.find_element_by_id("estimate_pickup_lng").get_attribute('value')
    # lat_entrega = browser.find_element_by_id("estimate_pickup_lat").get_attribute('value')
    # lng_entrega = browser.find_element_by_id("estimate_pickup_lng").get_attribute('value')

    geolocator = Nominatim(user_agent="sanico")
    coordenadas_retiro = geolocator.geocode(search["retiro"])
    coordenadas_entrega = geolocator.geocode(search["entrega"])
    lat_retiro = coordenadas_retiro.latitude
    lng_retiro = coordenadas_retiro.longitude
    lat_entrega = coordenadas_entrega.latitude
    lng_entrega = coordenadas_entrega.longitude

    coords_1 = (lat_retiro, lng_retiro)
    coords_2 = (lat_entrega, lng_entrega)
    km = distance.geodesic(coords_1, coords_2).km

    ### Ingresar tamaño de paquete:
    if search["tamano"]=="small":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_small'))
        )
        WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.ID, 'estimate_package_size_small'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[1]").click()
    elif search["tamano"]=="medium":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_medium'))
        )
        WebDriverWait(browser, timeout).until(
            EC.element_to_be_clickable((By.ID, 'estimate_package_size_medium'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[2]").click()
    elif search["tamano"]=="large":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_large'))
        )
        WebDriverWait(browser, timeout).until(
            EC.element_to_be_clickable((By.ID, 'estimate_package_size_large'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[3]").click()
    else:
        raise Exception('Valor no válido')

    ### Ingresar cantidad de paquetes:
    if int(search["cantidad_de_paquetes"])>=0 and int(search["cantidad_de_paquetes"])<=10:
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_number_of_packages'))
        )
        cantidad_de_paquetes = browser.find_element_by_id("estimate_number_of_packages")
        cantidad_de_paquetes.send_keys(str(search["cantidad_de_paquetes"]))
    else:
        raise Exception('Valor no válido')

    ### Ingresar opción de entrega:
    if search["opcion_de_entrega"]=="express":
        browser.find_element_by_xpath("//div[@id='estimate_form_container']/div[2]/div[14]/div/label[1]").click()
    elif search["opcion_de_entrega"]=="coordinado":
        browser.find_element_by_xpath("//div[@id='estimate_form_container']/div[2]/div[14]/div/label[2]").click()
    else:
        raise Exception('Valor no válido')

    ### Ingresar cantidad de asistentes:
    if int(search["cantidad_de_asistentes"])>=0 and int(search["cantidad_de_asistentes"])<=5:
        cantidad_de_asistentes = browser.find_element_by_id("estimate_laborer_number")
        cantidad_de_asistentes.send_keys(search["cantidad_de_asistentes"])
    else:
        raise Exception('Valor no válido')

    time.sleep(1)
    precio_txt = browser.find_element_by_id("estimate_value_label").text
    precio = re.search("(?<=\$)\s(.*?)\s(?=IVA)",precio_txt)[1]
    resultado = [lat_retiro, lng_retiro, lat_entrega, lng_entrega, km, precio]
    print(resultado)
    browser.quit()
    return resultado


### Lectura de datos de entrada

In [19]:
path = "./combinations_address.csv"
datos_entrada = pd.read_csv(path)
datos_entrada.head()

,retiro,entrega,tamano,opcion_de_entrega,cantidad_de_paquetes,cantidad_de_asistentes
0,Brito del pino 1491 Montevideo Uruguay,"Eulogio Caballero, Montevideo Uruguay",small,express,2,1
1,Brito del pino 1491 Montevideo Uruguay,"19 de abril 1213, Montevideo Uruguay",small,express,2,1
2,Brito del pino 1491 Montevideo Uruguay,"Copacabana 6808, Montevideo Uruguay",small,express,2,1
3,Brito del pino 1491 Montevideo Uruguay,"Galicia 1224, Montevideo Uruguay",small,express,2,1
4,Brito del pino 1491 Montevideo Uruguay,"Doctor Prudencio de Pena, Montevideo Uruguay",small,express,2,1


### Extracción de datos

In [20]:
inputs = []
resultados = []

for row in datos_entrada.iterrows():
    search = {}
    
    search["retiro"] = row[1]["retiro"]
    search["entrega"] = row[1]["entrega"]
    search["tamano"] = row[1]["tamano"]
    search["cantidad_de_paquetes"] = row[1]["cantidad_de_paquetes"]
    search["opcion_de_entrega"] = row[1]["opcion_de_entrega"]
    search["cantidad_de_asistentes"] = row[1]["cantidad_de_asistentes"]
    input_row = list(search.values())
    inputs.append(input_row)
    print(input_row)
    
    try:
        resultados.append(get_results(search))
    except:
        resultados.append([np.nan]*6)

resultados_df = pd.DataFrame(resultados)
resultados_df.head()

['Brito del pino 1491 Montevideo Uruguay', 'Eulogio Caballero, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9032725, -56.1570123, -34.8535167, -56.2452851, 9.777323024913034, '1528']
['Brito del pino 1491 Montevideo Uruguay', '19 de abril 1213, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9032725, -56.1570123, -34.8443465, -56.250894, 10.78932207449151, '1253']
['Brito del pino 1491 Montevideo Uruguay', 'Copacabana 6808, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9032725, -56.1570123, -34.8838262, -56.0466473, 10.316145890570759, '1473']
['Brito del pino 1491 Montevideo Uruguay', 'Galicia 1224, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9032725, -56.1570123, -34.9002335, -56.1903221, 3.063031653965184, '423']
['Brito del pino 1491 Montevideo Uruguay', 'Doctor Prudencio de Pena, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9032725, -56.1570123, -34.8989865, -56.1578912, 0.48222136270565036, '314']
['Brito del pino 1491 Montevideo Uruguay', 'Rambla 

[-34.8535167, -56.2452851, -34.8819445, -56.1812809, 6.647886131987248, '1308']
['Eulogio Caballero, Montevideo Uruguay', 'Rizal, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8535167, -56.2452851, -34.9009529, -56.1352843, 11.350416512504975, '1692']
['Eulogio Caballero, Montevideo Uruguay', 'Chamizo, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8535167, -56.2452851, -34.8889171, -56.1212496, 12.001436219852733, '1692']
['19 de abril 1213, Montevideo Uruguay', 'Copacabana 6808, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.8443465, -56.250894, -34.8838262, -56.0466473, 19.18277008273805, '1692']
['19 de abril 1213, Montevideo Uruguay', 'Galicia 1224, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.8443465, -56.250894, -34.9002335, -56.1903221, 8.313262736908493, '1088']
['19 de abril 1213, Montevideo Uruguay', 'Doctor Prudencio de Pena, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.8443465, -56.250894, -34.8989865, -56.1578912, 10.442686083355609, '1253'

[-34.8838262, -56.0466473, -34.8889171, -56.1212496, 6.843054487962966, '1253']
['Galicia 1224, Montevideo Uruguay', 'Doctor Prudencio de Pena, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9002335, -56.1903221, -34.8989865, -56.1578912, 2.9673952551019247, '1033']
['Galicia 1224, Montevideo Uruguay', 'Rambla República de México, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9002335, -56.1903221, -34.8941222, -56.0595102, 11.975703405054587, '1198']
['Galicia 1224, Montevideo Uruguay', 'Ricaldoni, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9002335, -56.1903221, -34.8938637, -56.1591115, 2.9389437008729415, '478']
['Galicia 1224, Montevideo Uruguay', 'Doctor Alberto Zubiría, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9002335, -56.1903221, -34.8434359, -56.1924935, 6.3041520694375395, '1198']
['Galicia 1224, Montevideo Uruguay', 'Adolfo berro, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.9002335, -56.1903221, -34.8642481, -56.2077112, 4.29703184702891

[-34.8941222, -56.0595102, -34.8681349, -56.1491542, 8.687566104656845, '1253']
['Rambla República de México, Montevideo Uruguay', 'Carlos Tellier, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8941222, -56.0595102, -34.8535167, -56.2452851, 17.57224904730428, '1637']
['Rambla República de México, Montevideo Uruguay', 'Cooper, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8941222, -56.0595102, -34.8700236, -56.0694155, 2.8226546974077236, '1473']
['Rambla República de México, Montevideo Uruguay', 'Oyarvide, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8941222, -56.0595102, -34.8542084, -56.1264252, 7.552179025996583, '1528']
['Rambla República de México, Montevideo Uruguay', 'Avenida Sarmiento, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8941222, -56.0595102, -34.9153352, -56.1673733, 10.135031483577231, '423']
['Rambla República de México, Montevideo Uruguay', 'Líber Arce, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8941222, -56.0595102, -34.898

[-34.8434359, -56.1924935, -34.8889171, -56.1212496, 8.239788601999264, '1308']
['Adolfo berro, Montevideo Uruguay', 'Los Nogales, Montevideo Uruguay', 'small', 2, 'express', 1]
[-34.8642481, -56.2077112, -34.8029901, -56.1143293, 10.915437622993213, '753']
['Adolfo berro, Montevideo Uruguay', 'Paysandú, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8642481, -56.2077112, -34.901975, -56.1895067, 4.504095683928287, '1088']
['Adolfo berro, Montevideo Uruguay', 'Himalaya, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8642481, -56.2077112, -34.8681349, -56.1491542, 5.3715833550395935, '1198']
['Adolfo berro, Montevideo Uruguay', 'Carlos Tellier, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8642481, -56.2077112, -34.8535167, -56.2452851, 3.6363386664519917, '1198']
['Adolfo berro, Montevideo Uruguay', 'Cooper, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8642481, -56.2077112, -34.8700236, -56.0694155, 12.661355279993716, '1582']
['Adolfo berro, Montevideo Urugu

[-34.8681349, -56.1491542, -34.8902774, -56.1800759, 3.7450871988238887, '478']
['Himalaya, Montevideo, Uruguay', 'Ignacio Barrios, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8681349, -56.1491542, -34.8562398, -56.1393665, 1.5944949015751693, '979']
['Himalaya, Montevideo, Uruguay', 'Escalada, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8681349, -56.1491542, -34.8525727, -56.2005685, 5.008441048886881, '1198']
['Himalaya, Montevideo, Uruguay', 'Fernando Torres, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8681349, -56.1491542, -34.8485567, -56.233298, 7.995216956185032, '753']
['Himalaya, Montevideo, Uruguay', 'Carlos Reyles, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8681349, -56.1491542, -34.8819445, -56.1812809, 3.312769106320536, '1088']
['Himalaya, Montevideo, Uruguay', 'Rizal, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8681349, -56.1491542, -34.9009529, -56.1352843, 3.855246219884912, '1143']
['Himalaya, Montevideo, Uruguay', 'Chamiz

[-34.8988864, -56.1440372, -34.8525727, -56.2005685, 7.287751288039386, '1308']
['Líber Arce, Montevideo, Uruguay', 'Fernando Torres, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8988864, -56.1440372, -34.8485567, -56.233298, 9.888200272482823, '1473']
['Líber Arce, Montevideo, Uruguay', 'Carlos Reyles, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8988864, -56.1440372, -34.8819445, -56.1812809, 3.8887968042890164, '1143']
['Líber Arce, Montevideo, Uruguay', 'Rizal, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8988864, -56.1440372, -34.9009529, -56.1352843, 0.8322081599861092, '924']
['Líber Arce, Montevideo, Uruguay', 'Chamizo, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8988864, -56.1440372, -34.8889171, -56.1212496, 2.3583358467043576, '1033']
['Hocquart, Montevideo, Uruguay', 'Ignacio Barrios, Montevideo, Uruguay', 'small', 2, 'express', 1]
[-34.8902774, -56.1800759, -34.8562398, -56.1393665, 5.302080712854402, '588']
['Hocquart, Montevideo, Uruguay'

,0,1,2,3,4,5
0,-34.903272,-56.157012,-34.853517,-56.245285,9.777323,1528
1,-34.903272,-56.157012,-34.844347,-56.250894,10.789322,1253
2,-34.903272,-56.157012,-34.883826,-56.046647,10.316146,1473
3,-34.903272,-56.157012,-34.900233,-56.190322,3.063032,423
4,-34.903272,-56.157012,-34.898986,-56.157891,0.482221,314


### Renombrar columnas

In [21]:
resultados_df.columns = ["lat_retiro",
                         "lng_retiro",
                         "lat_entrega",
                         "lng_entrega",
                         "km",
                         "precio"]
resultados_df.head()

,lat_retiro,lng_retiro,lat_entrega,lng_entrega,km,precio
0,-34.903272,-56.157012,-34.853517,-56.245285,9.777323,1528
1,-34.903272,-56.157012,-34.844347,-56.250894,10.789322,1253
2,-34.903272,-56.157012,-34.883826,-56.046647,10.316146,1473
3,-34.903272,-56.157012,-34.900233,-56.190322,3.063032,423
4,-34.903272,-56.157012,-34.898986,-56.157891,0.482221,314


### Combinar datos de entrada con resultados

In [22]:
results = pd.concat([datos_entrada, resultados_df], axis = 1)
results

,retiro,entrega,tamano,opcion_de_entrega,cantidad_de_paquetes,cantidad_de_asistentes,lat_retiro,lng_retiro,lat_entrega,lng_entrega,km,precio
0,Brito del pino 1491 Montevideo Uruguay,"Eulogio Caballero, Montevideo Uruguay",small,express,2,1,-34.903272,-56.157012,-34.853517,-56.245285,9.777323,1528
1,Brito del pino 1491 Montevideo Uruguay,"19 de abril 1213, Montevideo Uruguay",small,express,2,1,-34.903272,-56.157012,-34.844347,-56.250894,10.789322,1253
2,Brito del pino 1491 Montevideo Uruguay,"Copacabana 6808, Montevideo Uruguay",small,express,2,1,-34.903272,-56.157012,-34.883826,-56.046647,10.316146,1473
3,Brito del pino 1491 Montevideo Uruguay,"Galicia 1224, Montevideo Uruguay",small,express,2,1,-34.903272,-56.157012,-34.900233,-56.190322,3.063032,423
4,Brito del pino 1491 Montevideo Uruguay,"Doctor Prudencio de Pena, Montevideo Uruguay",small,express,2,1,-34.903272,-56.157012,-34.898986,-56.157891,0.482221,314
...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Fernando Torres, Montevideo, Uruguay","Rizal, Montevideo, Uruguay",small,express,2,1,-34.848557,-56.233298,-34.900953,-56.135284,10.681262,1582
296,"Fernando Torres, Montevideo, Uruguay","Chamizo, Montevideo, Uruguay",small,express,2,1,-34.848557,-56.233298,-34.888917,-56.121250,11.180703,1528
297,"Carlos Reyles, Montevideo, Uruguay","Rizal, Montevideo, Uruguay",small,express,2,1,-34.881945,-56.181281,-34.900953,-56.135284,4.703676,1198
298,"Carlos Reyles, Montevideo, Uruguay","Chamizo, Montevideo, Uruguay",small,express,2,1,-34.881945,-56.181281,-34.888917,-56.121250,5.542023,643


In [23]:
results.to_csv("results.csv")